This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ZKC19940412/water_ice_nep/colab-examples/example-2.ipynb)

In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Hardware accelerator` > `GPU`.

# Install TDMDpy from Source ($\sim$ 2 min)

In [ ]:
%%capture
%cd /content/
# Fetch tdmdpy repo
! git clone https://github.com/ZKC19940412/tdmdpy.git

# Install tdmdpy from source
%cd tdmdpy
! pip install .

# Install [GPUMD](https://github.com/brucefan1983/GPUMD) from Source ($\sim$ 2 min)
- More instructions can be found : https://gpumd.org/installation.html

In [ ]:
%%capture
%cd /content/
! git clone https://github.com/brucefan1983/GPUMD.git
%cd /content/GPUMD/src/
! make -j 8
! echo "GPUMD installation finishes!"
%cd /content/

# Clean up Workspace and Fetch Nessary Files

In [ ]:
%cd /content/
! git clone https://github.com/ZKC19940412/water_ice_nep.git
! rm -r sample_data

# Fetch Experimental Reference from Remote
- https://github.com/BingqingCheng/neural-network-potential-for-water-revPBE0-D3

In [ ]:
# Fetch from github repo
! git clone https://github.com/BingqingCheng/neural-network-potential-for-water-revPBE0-D3.git
! mkdir experimental_rdf/

# Copy experimental references to folder and clean up workspace
! cp -r /content/neural-network-potential-for-water-revPBE0-D3/radial-distribution-functions/exp-* experimental_rdf/
! mv experimental_rdf/exp-goo-T295.1K experimental_rdf/exp-goo-T295.1K.dat
! rm -r neural-network-potential-for-water-revPBE0-D3/

# Import Necessary Packages

In [ ]:
from ase.io import read, write
import mdtraj as mdt
import numpy as np
from pynvml import *
from pylab import *
from tdmdpy.atom_manipulate import decompose_dump_xyz
from tdmdpy.atom_manipulate import load_with_cell
from tdmdpy.create_systems import generate_water_box
from tdmdpy.thermodynamic_properties import compute_all_rdfs
from tdmdpy.thermodynamic_properties import get_block_average_quantities
from tdmdpy import vdos

# Custom Function Define

In [ ]:
def set_fig_properties(ax_list):
    tl = 6
    tw = 2
    tlm = 4

    for ax in ax_list:
        ax.tick_params(which='major', length=tl, width=tw)
        ax.tick_params(which='minor', length=tlm, width=tw)
        ax.tick_params(which='both', axis='both',
                       direction='in', right=True, top=True,
                       left=True)


def get_normalized_VDOS(data):
    frequency = data[:, 0]
    raw_intensity = data[:, 1]

    integral_val = np.trapz(y=raw_intensity, x=frequency)
    normalized_data = np.zeros_like(data)
    normalized_data[:, 0] = frequency
    normalized_data[:, 1] = raw_intensity / integral_val
    return normalized_data

# Obtain Information about GPU Architecture for Particular colab Instance

In [ ]:
if __name__ == "__main__":

    # Initialize nvml object
    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()

    # Loop through all avaliable devices
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))

# Perform Simulations for Run-time Profiling

## Create Liquid Water in Cubic Box

In [ ]:
if __name__ == "__main__":
    generate_water_box(target_density = 0.994,
                       number_of_molecules=512,
                       is_pre_equilibrate=False)

## Compose run.in

In [ ]:
%%writefile run.in
# NEP potential
potential /content/water_ice_nep/nep-pre-train-model/nep.txt

# time step for 1 fs
time_step 1

# Initialize velocity at 298K
velocity 298

# Run NVE
ensemble nve

# run 1000 steps, equal to 1 ps simulation
run 1000

## Peform Simualtions ($\sim$ 7 s)

In [ ]:
! /content/GPUMD/src/gpumd < run.in > run_time.log
! grep "Time used ="  run_time.log > grepped_run_time.out

## Compute Run-time in ns/day

In [ ]:
if __name__ == "__main__":

    # Load run time in ps per scond
    run_time_in_ps_per_sec = np.loadtxt('grepped_run_time.out', usecols=3) ** (-1)

    # Convert run time from ps/s to ns/day
    # 1000 ps = 1 ns
    # 1 day = 24 hours = 86400 seconds
    run_time_in_ns_per_day = (24.0 * 60.0 * 60.0 / 1000.0)  * run_time_in_ps_per_sec
    print('Run Time Performance : %.2f  ns/day' % run_time_in_ns_per_day)

# Perform Simulations for RDFs and Hydrogen-vDOS

## Compose run.in

In [ ]:
%%writefile run.in
# Set up NEP potential
potential /content/water_ice_nep/nep-pre-train-model/nep.txt

# time step for 0.5 fs
time_step 0.5

# Initialize velocity at 300K
velocity 300

# Run NPT equalibration with SCR method for 300K Tini and Tend,
# and 100 for Tcoupling, 0 bar for pressures, and
# 2 Gpa for pressure coupling and 1000 steps
ensemble   npt_scr 300 300 100 0 0 0 2 2 2 1000

# run 8000 steps, equal to 4 ps simulation
run 8000

# Run NVT prodution with BDP thermostats for 300K Tini and Tend,
# and 100 for Tcoupling
ensemble nvt_bdp 300 300 100


# dump extended xyz with every steps, dump force and velocity too
# Critcal for vDOS
dump_exyz 1 1 1

# dump themodynamic quantities every steps
dump_thermo 1

# Run 100000 steps, equal to 50 ps simulation
run 100000

## Peform Simualtions ($\sim$ 20 min)
- 8000 and 100000 steps used here only serve as illustration purpose.

In [ ]:
! /content/GPUMD/src/gpumd < run.in

## Visualize T vs Time from Production Run ($\sim$ 5 min)

In [ ]:
if __name__ == "__main__":

    #  Set up Figure Styles
    aw = 2
    fs = 22
    font = {'size': fs}
    matplotlib.rc('font', **font)
    matplotlib.rc('axes', linewidth=aw)

    # Truncate dump.xyz for faster runtime
    write('dump_rdf.xyz', read('dump.xyz', index='::100'))
    write('dump_vdos.xyz', read('dump.xyz', index=':10000'))

     # Load in temperature data from thermo.out
    data = np.loadtxt('thermo.out')

    # Extract box dimension from thermo.out
    # length scale goes from angstrom to nm
    cell_length_matrix = data[:, -3:] / 10.0
    cell_angle_matrix = 90 * np.ones_like(cell_length_matrix)

    # Denote time step and sample rate
    time_step = 5e-4
    sample_rate = 1

    # Derive time span
    time_span = time_step * sample_rate * np.arange(0, len(data), 1)

    # Load temperature and compute its block averages
    temperature = data[:, 0]
    block_average_temperature = get_block_average_quantities(temperature,
                                                             n_block=5)

    print('Inspect Properties from NVT Simulations: ')
    print('Set point temperature: %.3f K' %np.round(
        block_average_temperature.mean(), 3))
    print('\n')
    figure()
    set_fig_properties([gca()])
    plot(time_span, temperature, label='T')
    plot(time_span, np.round(
        block_average_temperature.mean(), 3) * np.ones_like(
        time_span), 'r--',
         label=r'$T_{block \ avg}$', lw=3)
    xlabel('Time (ps)')
    ylabel('Temperature (K)')
    ylim([260, 340])
    legend(loc=4, fontsize=16)
    show()

## Compute RDFs

In [ ]:
if __name__ == "__main__":

    # Decompose dump.xyz
    decompose_dump_xyz('dump_rdf.xyz')

    # Inject Reference PDB file into the trajectory'
    pos_trajectory = load_with_cell('pos.xyz', cell_length_matrix[::100],
                                    cell_angle_matrix[::100],
                                    top='ini_pos.pdb')

    # Compute RDFS
    rdfs = compute_all_rdfs(pos_trajectory, n_bins=400, is_save_rdf=False)

    # Save RDFS for each pair, multiple by 10 to
    # go from nm to angstrom for length unit.
    np.savetxt('nep_goo_T_300K.out',
               np.vstack([rdfs['O-O'][0][:] * 10, rdfs['O-O'][1][:]]).T)
    np.savetxt('nep_goh_T_300K.out',
               np.vstack([rdfs['H-O'][0][:] * 10, rdfs['H-O'][1][:]]).T)
    np.savetxt('nep_ghh_T_300K.out',
               np.vstack([rdfs['H-H'][0][:] * 10, rdfs['H-H'][1][:]]).T)

## Compute Hydorgen-VDOS ($\sim$ 8 min)

In [ ]:
if __name__ == "__main__":

    # Decompose dump.xyz
    decompose_dump_xyz('dump_vdos.xyz')

    # Inject Reference PDB file into the trajectory'
    vel_trajectory = load_with_cell('vel.xyz', cell_length_matrix[:10000],
                                    cell_angle_matrix[:10000],
                                    top='ini_vel.pdb')
    # Compute VDOS
    vdos = vdos.compute_all_vdos(vel_trajectory, dt=0.5, Dt=2000.0)

    # Save VDOS for each species.
    np.savetxt('nep_vdos_h_T_300K.out',
               np.vstack([vdos['H'][0][:],
                          vdos['H'][1][:]]).T)

## Visualize RDFs & H-vDOS for Liquid Water

In [ ]:
if __name__ == "__main__":


    # Load in experimental data
    rdf_exp_goo_T_295_pt1K = np.loadtxt(
        'experimental_rdf/exp-goo-T295.1K.dat',skiprows=6)
    rdf_exp_goh_T_300K = np.loadtxt(
        'experimental_rdf/exp-goh-T300K.dat', skiprows=2)
    rdf_exp_ghh_T_300K = np.loadtxt(
        'experimental_rdf/exp-ghh-T300K.dat', skiprows=2)

     # Load in nep rdf data
    rdf_nep_goo_T_300K = np.loadtxt('nep_goo_T_300K.out')
    rdf_nep_goh_T_300K = np.loadtxt('nep_goh_T_300K.out')
    rdf_nep_ghh_T_300K = np.loadtxt('nep_ghh_T_300K.out')

    vdos_data = get_normalized_VDOS(np.loadtxt('nep_vdos_h_T_300K.out'))

    figure(figsize=(12, 10))
    subplot(2, 2, 1)
    set_fig_properties([gca()])
    plot(rdf_exp_goo_T_295_pt1K[:, 0],
         rdf_exp_goo_T_295_pt1K[:, 1], lw=3, ls='--',
         c='k')
    plot(rdf_nep_goo_T_300K[:, 0], rdf_nep_goo_T_300K[:, 1],
         lw=2, c='r', ls='-')
    text(0.005, 0.9, '(a)', c='k', fontsize=14,
         transform=gca().transAxes, fontweight='bold')
    ylabel(r'$g_{OO}(r)$', fontsize=22)
    xlabel(r'r ($\AA$)', fontsize=22)
    xlim([0, 6])
    ylim([-0.1, 3])
    gca().set_yticks(np.linspace(0, 3, 4))
    gca().set_xticks(np.linspace(0, 6, 7))

    subplot(2, 2, 2)
    set_fig_properties([gca()])
    plot(rdf_exp_goh_T_300K[:, 0], rdf_exp_goh_T_300K[:, 1],
         lw=3, ls='--', c='k')
    plot(rdf_nep_goh_T_300K[:, 0], rdf_nep_goh_T_300K[:, 1],
         lw=2, c='r', ls='-')
    text(0.005, 0.9, '(b)', c='k', fontsize=14,
         transform=gca().transAxes, fontweight='bold')
    ylabel(r'$g_{OH}(r)$', fontsize=22)
    xlabel(r'r ($\AA$)', fontsize=22)
    xlim([0, 6])
    gca().set_yticks(np.linspace(0, 3, 4))
    ylim([-0.1, 2.5])
    gca().set_xticks(np.linspace(0, 6, 7))

    subplot(2, 2, 3)
    set_fig_properties([gca()])
    plot(rdf_exp_ghh_T_300K[:, 0], rdf_exp_ghh_T_300K[:, 1],
         lw=3, ls='--', c='k', label='Experiment')
    plot(rdf_nep_ghh_T_300K[:, 0], rdf_nep_ghh_T_300K[:, 1],
         lw=2, c='r', ls='-', label='NEP')
    text(0.005, 0.9, '(c)', c='k', fontsize=14,
         transform=gca().transAxes, fontweight='bold')
    ylabel(r'$g_{HH}(r)$', fontsize=22)
    xlabel(r'r ($\AA$)', fontsize=22)
    xlim([0, 6])
    gca().set_yticks(np.linspace(0, 3, 4))
    ylim([-0.1, 3.7])
    gca().set_xticks(np.linspace(0, 6, 7))
    legend(loc='best', frameon=False)

    subplot(2, 2, 4)
    set_fig_properties([gca()])
    plot(vdos_data[:,0], vdos_data[:,1]/vdos_data[:,1].max(),
         lw=2, c='r', label='NEP')
    text(0.008, 0.9, '(d)', c='k', fontsize=14,
         transform=gca().transAxes, fontweight='bold')
    gca().vlines(x=615, ymax=2.5, ymin=-0.05,
                 color='k', ls='--', lw=3)
    gca().vlines(x=1646, ymax=2.5, ymin=-0.05,
                 color='k', ls='--', lw=3)
    gca().vlines(x=3406, ymax=2.5, ymin=-0.05,
                 color='k', ls='--', lw=3)
    gca().set_xticks(np.arange(0, 5000, 1000))
    xlim([0, 5000])
    xlabel(r'$\omega$ ($cm^{-1}$)', fontsize=22)
    ylabel('vDOS (arb.unit.)', fontsize=22)
    gca().set_yticks(np.arange(0, 1.6, 0.4))
    gca().set_yticklabels([])
    ylim([-0.05, 1.6])
    subplots_adjust(hspace=0.3, wspace=0.2)
    savefig('structural-proper-LW-300K.png', dpi=300)
    show()